In [44]:
import pysal.esda.mapclassify as mapclassify
import pandas as pd
from mapboxgl.utils import *
from mapboxgl.viz import *
from sklearn.cluster import KMeans

In [48]:
df = pd.read_csv('data/voters.csv')
df.columns = ['Name', 'Address', 'lat', 'lon', 'zipcode']
df.head(5)

,Name,Address,lat,lon,zipcode
0,Daragh Parden,10353 GOLD COAST PL,32.905949,-117.131853,130710
1,Danaly Dostie,10460 BAYWOOD AVE,32.908405,-117.129932,130710
2,Shakilya Shellito,8845 PAGODA WAY,32.906215,-117.133823,130710
3,Dakori Kanai,10437 STANFIELD CIR,32.907817,-117.132717,130710
4,Traciann Cheirs,8766 BELFAST CIR,32.907334,-117.135225,130710


In [60]:
def create_clusters(dataframe, volunteers):
    kmeans = KMeans(n_clusters=volunteers).fit(dataframe)
    return kmeans

def get_centroids(kmeans):
    return kmeans.cluster_centers_

def get_mappings(kmeans):
    return kmeans.labels_

kmeans = create_clusters(df[['lat','lon']], 3)
df['cluster'] = get_mappings(kmeans)

In [61]:
token = 'pk.eyJ1IjoidWJhY2hyaXMiLCJhIjoiY2sxYjczdWhpMGZuMzNjb2I5OGlqb3gwaCJ9.iLrtxaVXfhsJM0iyWwdQ5Q'

In [62]:
df_to_geojson(df, filename='points1.geojson', 
                    properties = ['Name','Address','zipcode','cluster'],
                     lat='lat', lon='lon', precision=3)

{'feature_count': 13597, 'filename': 'points1.geojson', 'type': 'file'}

In [64]:
color_breaks = mapclassify.Natural_Breaks(df['cluster'], k=3, initial=0).bins
color_stops = create_color_stops(color_breaks, colors='YlGnBu')

# Create the viz from the dataframe
viz = CircleViz('points1.geojson',
                access_token=token, 
                height='1000px',
                color_property = "cluster",
                color_stops = color_stops,
                center = (-117.176, 32.8560),
                zoom = 11,
                below_layer = 'waterway-label')

viz.show()